In [1]:
import pandas as pd
import ast
import time

In [2]:
import os
import sys

current_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, ".."))
sys.path.insert(0, parent_dir)

from src.orcid_data_utilities import get_author_info_by_orcid

ImportError: cannot import name 'get_author_info_by_orcid' from 'src.orcid_data_utilities' (/Users/saraborello/Documents/GITHUB/Data_Semantics/KB_data_science/src/orcid_data_utilities.py)

In [3]:
df_papers = pd.read_csv("../data/papers/papers.csv")
df_papers

,doi,year,authors,author_orcids,type,topics,title
0,10.1016/j.nut.2022.111687,2022,"['Marco Giani', 'Emanuele Rezoagli', 'Alice Gr...","['0000-0001-8048-2721', None, None, '0000-0002...",journal-article,Nutrition and Health in Aging,Low skeletal muscle index and myosteatosis as ...
1,10.3390/cancers13071634,2021,"['Lavinia Monaco', 'Maria Gemelli', 'Irene Got...","['0000-0002-3954-6989', '0000-0003-1143-0252',...",journal-article,Lung Cancer Treatments and Mutations,Metabolic Parameters as Biomarkers of Response...
2,10.1016/j.jhep.2023.09.008,2023,"['Riccardo De Carlis', 'Andrea Lauterio', 'Dav...","['0000-0003-3697-1653', '0000-0001-6110-8582',...",journal-article,Organ Transplantation Techniques and Outcomes,Reply to: “Excellent long-term outcomes after ...
3,10.1016/j.ijantimicag.2024.107200,2024,"['Amedeo De Nicolò', 'Alice Palermiti', 'Stefa...","['0000-0002-5973-9948', '0000-0003-0860-280X',...",journal-article,HIV Research and Treatment,"Plasma, intracellular and lymph node antiretro..."
4,10.3390/jpm12101587,2022,"['Alessio Gerussi', 'Damiano Verda', 'Claudio ...","['0000-0002-5086-0514', None, '0000-0003-3301-...",journal-article,Liver Diseases and Immunity,LLM-PBC: Logic Learning Machine-Based Explaina...
...,...,...,...,...,...,...,...
276,10.1109/wi-iat59888.2023.00066,2023,"['Stefano Locci', 'Luigi Di Caro', 'Giovanni L...","['0009-0006-9725-2045', '0000-0002-7570-637X',...",proceedings-article,Privacy-Preserving Technologies in Data,Explainability of the Effects of Non-Perturbat...
277,10.1108/lht-12-2020-0317,2021,"['Federico Pianzola', 'Maurizio Toccu', 'Marco...","['0000-0001-6634-121X', '0000-0002-5769-1311',...",journal-article,Impact of Technology on Adolescents,Readers' engagement through digital social rea...
278,10.1007/978-3-031-88714-7_34,2025,"['Luca Herranz-Celotti', 'Blessing Guembe', 'G...","['0000-0003-2460-8099', '0000-0002-7576-9127',...",book-chapter,Privacy-Preserving Technologies in Data,Can Generative AI Adequately Protect Queries? ...
279,10.1007/978-3-031-88720-8_52,2025,"['Udo Kruschwitz', 'Marinella Petrocchi', 'Mar...","['0000-0002-5503-0341', '0000-0003-0591-877X',...",book-chapter,Misinformation and Its Impacts,ROMCIR 2025: Overview of the 5th Workshop on R...


In [5]:
import requests

def get_author_info_by_orcid(orcid):
    search_url = f"https://api.openalex.org/authors?filter=orcid:{orcid}"
    try:
        response = requests.get(search_url, timeout=10)
        response.raise_for_status()
    except requests.exceptions.Timeout:
        print(f"Timeout per ORCID {orcid}")
        return {}
    except requests.exceptions.RequestException as e:
        print(f"Errore per ORCID {orcid}: {e}")
        return {}

    result = {}
    ins_id = []
    ins_name = []
    fields = []

    if response.status_code == 200:
        data = response.json()
        results = data.get('results', [])

        for author in results:
            affiliations = author.get('affiliations', [])
            for institution in affiliations:
                ins = institution.get('institution', {})
                ins_id.append(ins.get('id'))
                ins_name.append(ins.get('display_name'))

            topics = author.get("topics", [])
            for t in topics:
                field = t.get("display_name")
                if field and field not in fields:
                    fields.append(field)

            result["id"] = author.get("id")
            result["Name"] = author.get("display_name")
            result["ins_id"] = ins_id
            result["ins_name"] = ins_name
            result["topics"] = fields
            result["hindex"] = author.get("summary_stats", {}).get("h_index", 0)

            break  # Solo il primo

        if not result:
            print(f"Nessun autore affiliato trovato per ORCID: {orcid}")

    else:
        print(f"Errore nella richiesta: {response.status_code}")

    return result

In [6]:
orcids_bicocca = []

df_bicocca = pd.read_csv("../data/authors/authors_internal.csv")

for index, row in df_bicocca.iterrows():
    orcid = row["orcid"]
    orcids_bicocca.append(orcid)

In [7]:
df_authors_external = pd.DataFrame()

In [8]:
orcids = []

for index, row in df_papers.iterrows():
    ids = ast.literal_eval(row["author_orcids"]) if isinstance(row["author_orcids"], str) else row["author_orcids"]
    for i in ids:
        if i and i not in orcids and i not in orcids_bicocca:
            orcids.append(i)

In [10]:
for orcid in orcids:
    print(f"ricerca per orcid {orcid}")
    author_info = get_author_info_by_orcid(orcid)
    if author_info:
        df_author = pd.DataFrame([author_info])
        df_author.loc[0, "orcid"] = orcid
    else:
        continue
    df_authors_external = pd.concat([df_authors_external, df_author], ignore_index=True)
    time.sleep(5)

ricerca per orcid 0000-0001-8048-2721
ricerca per orcid 0000-0002-1833-6587
ricerca per orcid 0000-0002-9721-6304
ricerca per orcid 0000-0003-0718-2321
ricerca per orcid 0000-0002-3954-6989
ricerca per orcid 0000-0003-1143-0252
ricerca per orcid 0000-0002-1937-9116
ricerca per orcid 0000-0003-3690-8582
ricerca per orcid 0000-0001-7611-7369
ricerca per orcid 0000-0002-1659-788X
ricerca per orcid 0000-0001-8432-5408
ricerca per orcid 0000-0003-4197-2060
ricerca per orcid 0000-0003-3697-1653
ricerca per orcid 0000-0001-6110-8582
ricerca per orcid 0000-0002-3916-3566
ricerca per orcid 0000-0002-5973-9948
ricerca per orcid 0000-0003-0860-280X
ricerca per orcid 0000-0002-1321-4126
ricerca per orcid 0000-0002-0375-9990
ricerca per orcid 0000-0002-3601-7686
ricerca per orcid 0000-0002-2673-2731
ricerca per orcid 0000-0003-2534-8815
ricerca per orcid 0000-0002-5086-0514
ricerca per orcid 0000-0003-3301-2967
ricerca per orcid 0000-0002-9999-2331
ricerca per orcid 0000-0003-3262-1998
ricerca per 

In [20]:
df_authors_external

,id,Name,ins_id,ins_name,topics,hindex,orcid
0,https://openalex.org/A5062159833,Marco Giani,"[https://openalex.org/I66752286, https://opena...","[University of Milano-Bicocca, Azienda Ospedal...","[Mechanical Circulatory Support Devices, Respi...",27,0000-0001-8048-2721
1,https://openalex.org/A5062159833,Marco Giani,"[https://openalex.org/I66752286, https://opena...","[University of Milano-Bicocca, Azienda Ospedal...","[Mechanical Circulatory Support Devices, Respi...",27,0000-0001-8048-2721
2,https://openalex.org/A5062159833,Marco Giani,"[https://openalex.org/I66752286, https://opena...","[University of Milano-Bicocca, Azienda Ospedal...","[Mechanical Circulatory Support Devices, Respi...",27,0000-0001-8048-2721
3,https://openalex.org/A5062159833,Marco Giani,"[https://openalex.org/I66752286, https://opena...","[University of Milano-Bicocca, Azienda Ospedal...","[Mechanical Circulatory Support Devices, Respi...",27,0000-0001-8048-2721
4,https://openalex.org/A5021090961,Marco Porta,"[https://openalex.org/I4210091499, https://ope...","[Istituti Clinici Scientifici Maugeri, Univers...","[Myasthenia Gravis and Thymoma, Shoulder Injur...",9,0000-0002-1833-6587
...,...,...,...,...,...,...,...
108,https://openalex.org/A5033757788,Elton Dajti,"[https://openalex.org/I4210097083, https://ope...","[Policlinico S.Orsola-Malpighi, University of ...","[Liver Disease Diagnosis and Treatment, Liver ...",19,0000-0003-2905-1146
109,https://openalex.org/A5033757788,Elton Dajti,"[https://openalex.org/I4210097083, https://ope...","[Policlinico S.Orsola-Malpighi, University of ...","[Liver Disease Diagnosis and Treatment, Liver ...",19,0000-0003-2905-1146
110,https://openalex.org/A5033757788,Elton Dajti,"[https://openalex.org/I4210097083, https://ope...","[Policlinico S.Orsola-Malpighi, University of ...","[Liver Disease Diagnosis and Treatment, Liver ...",19,0000-0003-2905-1146
111,https://openalex.org/A5052311759,M. De Angelis,"[https://openalex.org/I4210165206, https://ope...","[Azienda Ospedaliera Carlo Poma, University of...","[Liver Disease Diagnosis and Treatment, Hepato...",12,0000-0001-7759-1185


In [21]:
df_authors_external_no_duplicate = df_authors_external[~df_authors_external.duplicated(subset=["orcid"], keep="first")]
df_authors_external_no_duplicate

,id,Name,ins_id,ins_name,topics,hindex,orcid
0,https://openalex.org/A5062159833,Marco Giani,"[https://openalex.org/I66752286, https://opena...","[University of Milano-Bicocca, Azienda Ospedal...","[Mechanical Circulatory Support Devices, Respi...",27,0000-0001-8048-2721
4,https://openalex.org/A5021090961,Marco Porta,"[https://openalex.org/I4210091499, https://ope...","[Istituti Clinici Scientifici Maugeri, Univers...","[Myasthenia Gravis and Thymoma, Shoulder Injur...",9,0000-0002-1833-6587
6,https://openalex.org/A5001598786,Simone Famularo,"[https://openalex.org/I4210109354, https://ope...",[Institut de Recherche contre les Cancers de l...,[Hepatocellular Carcinoma Treatment and Progno...,20,0000-0002-9721-6304
14,https://openalex.org/A5016366315,Alessandro Giani,"[https://openalex.org/I4210094195, https://ope...",[Azienda Socio Sanitaria Territoriale Grande O...,"[Pancreatic and Hepatic Oncology Research, Hep...",21,0000-0003-0718-2321
15,https://openalex.org/A5032994231,Lavinia Monaco,"[https://openalex.org/I4210110840, https://ope...","[Azienda Ospedaliera San Gerardo, Istituti di ...",[Radiomics and Machine Learning in Medical Ima...,6,0000-0002-3954-6989
16,https://openalex.org/A5090507377,M Gemelli,"[https://openalex.org/I4210153126, https://ope...",[Istituti di Ricovero e Cura a Carattere Scien...,"[Occupational and environmental lung diseases,...",10,0000-0003-1143-0252
35,https://openalex.org/A5038942338,Matteo Bauckneht,"[https://openalex.org/I83816512, https://opena...","[University of Genoa, Ospedale Policlinico San...","[Prostate Cancer Treatment and Research, Parki...",31,0000-0002-1937-9116
37,https://openalex.org/A5086247944,Carlo Genova,"[https://openalex.org/I4210146472, https://ope...","[Ospedale Policlinico San Martino, University ...","[Lung Cancer Treatments and Mutations, Cancer ...",37,0000-0003-3690-8582
38,https://openalex.org/A5053651576,Diego Cortinovis,"[https://openalex.org/I66752286, https://opena...","[University of Milano-Bicocca, Istituti di Ric...","[Lung Cancer Treatments and Mutations, Cancer ...",44,0000-0001-7611-7369
39,https://openalex.org/A5070152758,Lodovica Zullo,"[https://openalex.org/I2801843802, https://ope...","[Institut Gustave Roussy, University of Genoa,...","[Lung Cancer Treatments and Mutations, Cancer ...",12,0000-0002-1659-788X
